# PGCS Number Scraper

This notebook is designed to obtain all the neccesary information to complete create an image scraper.

## Get HTML

Get the HTML from the PCGS website.

### Imports

Import `cloudscraper` for scraping. Import `pandas` for data manipulation.

In [114]:
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
import numpy as np
import requests
import time
import matplotlib.pyplot as plt



### Headers and scraper

In [7]:
# Establish connectection and pull HTML
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:70.0) Gecko/20100101 Firefox/70.0',
         'referer': 'https://www.google.com/',
         'Upgrade-Insecure-Requests': '1'}
url = 'https://www.pcgs.com/prices/detail/mercury-dime/703/most-active/ms?pn=1&ps=-1'
scraper = cloudscraper.create_scraper()
status = scraper.get(url)
html = status.text 

print('Status code: ' + str(status.status_code))

Status code: 200


### Get table from PCGS

For now, we don't need the description as we originally thought. Might delete the manuel indexings later.

In [97]:
# Creates and cleans the table
df = pd.read_html(html)
df = pd.DataFrame(df[0],dtype=object)
df = df.drop([0,1])
df = df.rename(columns=df.iloc[0])
df = df[['PCGS #','Description']].copy()
df = df.rename(columns={'PCGS #': 'id', 'Description': 'description'})
df = df.dropna()
df = df.reset_index(drop=True)

# Manual indexing check if error occurs, pcgs might change something
df = df.drop(range(0,3))
df = df.drop(range(163,166))
df = df.set_index('id')
df['description'] = df['description'].str.replace('Shop with Affiliates ', '')
df['description'] = df['description'].str.replace(' Mercury', '')
df['description'] = df['description'].str.replace(' ', '')

# Display table
df

,description
id,
4904,1916
4905,1916
4906,1916-D
4907,1916-D
4908,1916-S
...,...
5057,1945
5058,1945-D
5059,1945-D


Store the data to a CSV for later use

In [9]:
# Outputs the DataFrame to a CSV
location = 'assets/pgcs_number_output.csv'
df.to_csv(location)

print('See assets folder for output')

See assets folder for output


### Generate URLs from DataFrame

In [10]:
all_date_urls = []
fixed_url = 'https://www.pcgs.com/coinfacts/coin/'
fixed_url_ending = '-10c/images/'

for id in df.index:
  all_date_urls.append(fixed_url + df.loc[id,'description'] + fixed_url_ending + id)

all_date_urls

['https://www.pcgs.com/coinfacts/coin/1916-10c/images/4904',
 'https://www.pcgs.com/coinfacts/coin/1916-10c/images/4905',
 'https://www.pcgs.com/coinfacts/coin/1916-D-10c/images/4906',
 'https://www.pcgs.com/coinfacts/coin/1916-D-10c/images/4907',
 'https://www.pcgs.com/coinfacts/coin/1916-S-10c/images/4908',
 'https://www.pcgs.com/coinfacts/coin/1916-S-10c/images/4909',
 'https://www.pcgs.com/coinfacts/coin/1917-10c/images/4910',
 'https://www.pcgs.com/coinfacts/coin/1917-10c/images/4911',
 'https://www.pcgs.com/coinfacts/coin/1917-D-10c/images/4912',
 'https://www.pcgs.com/coinfacts/coin/1917-D-10c/images/4913',
 'https://www.pcgs.com/coinfacts/coin/1917-S-10c/images/4914',
 'https://www.pcgs.com/coinfacts/coin/1917-S-10c/images/4915',
 'https://www.pcgs.com/coinfacts/coin/1918-10c/images/4916',
 'https://www.pcgs.com/coinfacts/coin/1918-10c/images/4917',
 'https://www.pcgs.com/coinfacts/coin/1918-D-10c/images/4918',
 'https://www.pcgs.com/coinfacts/coin/1918-D-10c/images/4919',
 'ht

In [22]:
#example of how to download images
'''
image = 'https://images.pcgs.com/CoinFacts/80811003_51807535_2200.jpg'

test_scraper = cloudscraper.create_scraper()
img_data = test_scraper.get(image).content
#print(img_data.status_code)

with open('assets/image_name.jpg', 'wb') as handler:
    handler.write(img_data)
    '''

In [111]:

def download_image(_url,_grade):
    image_scraper = cloudscraper.create_scraper()
    img_data = image_scraper.get(_url).content
    
    with open('assets/graded_images/' + str(_grade) +'/'+ _url[-19:], 'wb') as handler:
        handler.write(img_data)
     
    
    print('downloaded: ' + _url + ' to '+ 'assets/graded_images/' + str(_grade) +'/' + _url)

In [ ]:
# downloader for old dataset
'''
def download_image(_url,_folder,_name):
    image_scraper = cloudscraper.create_scraper()
    img_data = image_scraper.get(_url).content
    
    with open('assets/image_dataset/' + str(_folder) +'/'+ _name, 'wb') as handler:
        handler.write(img_data)
     
'''
#    if not os.path.isfile('assets/image_dataset/' + str(_folder) +'/'+ _name):
#        with open('assets/image_dataset/' + str(_folder) +'/'+ _name, 'wb') as handler:
#           handler.write(img_data)
'''       
    print('downloaded: ' + _url + ' to '+ 'assets/image_dataset/' + str(_folder) +'/' + _name)
    '''

In [12]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start
#see https://stackoverflow.com/questions/1883980/find-the-nth-occurrence-of-substring-in-a-string

In [31]:
def generate_all_image_urls(URL):
    scraper = cloudscraper.create_scraper()
    raw_scrape = scraper.get(URL)
    html = raw_scrape.text
    image_list = []
    
    for x in range (html.count('https://images.pcgs.com/CoinFacts/')):

        temp = html[find_nth(html,'https://images.pcgs.com/CoinFacts/', x):find_nth(html,'https://images.pcgs.com/CoinFacts/', x)+60]
    
        if temp[-1] != 'g':
            temp += 'g'
        image_list.append(temp)
        #for x in image_list:
            #print(x)
            

            
    return list(set(image_list))
    

In [99]:
generate_all_image_urls('https://www.pcgs.com/coinfacts/coin/1916-10c/images/4904')

['https://images.pcgs.com/CoinFacts/80811003_51807535_2200.jpg',
 'https://images.pcgs.com/CoinFacts/39198765_188265655_2200.jpg',
 'https://images.pcgs.com/CoinFacts/43556317_237699607_2200.jpg',
 'https://images.pcgs.com/CoinFacts/33274918_142794105_2200.jpg',
 'https://images.pcgs.com/CoinFacts/31487624_45111876_2200.jpg',
 'https://images.pcgs.com/CoinFacts/35646089_136163728_2200.jpg',
 'https://images.pcgs.com/CoinFacts/44915029_241590302_2200.jpg',
 'https://images.pcgs.com/CoinFacts/45953595_251032028_2200.jpg',
 'https://images.pcgs.com/CoinFacts/41478887_208495498_2200.jpg',
 'https://images.pcgs.com/CoinFacts/39163745_188938322_2200.jpg',
 'https://images.pcgs.com/CoinFacts/42908036_223280999_2200.jpg',
 'https://images.pcgs.com/CoinFacts/38796109_178831085_2200.jpg',
 'https://images.pcgs.com/CoinFacts/43736767_229466801_2200.jpg',
 'https://images.pcgs.com/CoinFacts/15673901_148983745_2200.jpg',
 'https://images.pcgs.com/CoinFacts/41435587_208806595_2200.jpg',
 'https://im

In [90]:
def generate_grade(URL):
    scraper = cloudscraper.create_scraper()
    raw_scrape = scraper.get(URL)
    html = raw_scrape.text
    grade_list = []
    for x in range (html.count('GRADE')):
        temp = html[find_nth(html, 'GRADE',x+1)+66:find_nth(html, 'GRADE',x+1)+68]
        

        #temp = html[find_nth(html,'https://images.pcgs.com/CoinFacts/', x):find_nth(html,'https://images.pcgs.com/CoinFacts/', x)+60]
    
        if temp[-1] == '<':
            temp = temp [:-1]
        grade_list.append(temp)
    #grade_list.pop(0)
    return grade_list

In [55]:
generate_grade('https://www.pcgs.com/coinfacts/coin/1944-D-10c/images/5052')

17


['68',
 '66',
 '66',
 '66',
 '65',
 '65',
 '62',
 '58',
 '58',
 '58',
 '53',
 '53',
 '45',
 '45',
 '40',
 '35',
 '15']

In [62]:
print(all_date_urls[0][-4:])

4904


In [113]:
for x in range(len(all_date_urls)):
    images = generate_all_image_urls(all_date_urls[x])
    grades = generate_grade(all_date_urls[x])
    #time.sleep(5)
    
    for y in range(len(images)):
        download_image(images[y],grades[y])

17


KeyboardInterrupt: 